<h2> Searched literature data preprocessing </h2> 

In [ ]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [ ]:
# import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import re
import time
import numpy as np
import numpy as np

<h3> Parameters: </h3>

In [ ]:
# columns of file: potential_related_literature.csv
columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
# e.g., ["10.1113/JP282626", "35851953", "PMC10087288", 
#        "Cortico-thalamocortical interactions for learning, memory and decision-making",
#        "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10087288/", "PMC",
#        "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10087288/pdf/TJP-601-25.pdf", "PMC"]

<h3> Predefined fucntions: </h3> 

In [ ]:
def preprocess_pubmed(source_path, output_path, start, end):
    print("Starting preprocessing search results from PubMed...")

    df = pd.read_csv(source_path, sep=',')
    df = df[["DOI", "PMID", "PMCID", "Title"]]
    
    for ind in range(start, end):
        # sleep to avoid to be blocked
        time.sleep(random.randint(3, 5))
        # if(ind%50 == 0):
        #     time.sleep(random.randint(10,15)*10)
        
        # request the webpage
        # the columns PMID, Title don't contain np.nan
        pmid = str(df["PMID"][ind]).strip()
        url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
        # proxies = plib.get_proxies()
        soup = plib.request_webpage(url)
        # print(soup)
        
        # get pmcid
        if df["PMCID"][ind] != df["PMCID"][ind]: # PMCID is np.nan
            try:
                pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
            except:
                pmcid = np.nan
        else: # PMCID is not np.nan
            pmcid = str(df["PMCID"][ind]).strip()
        # print(pmcid)

        # get doi
        if df["DOI"][ind] != df["DOI"][ind]: # DOI is np.nan
            try:
                doi = soup.find_all("span", {"class": "identifier doi"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
            except:
                doi  = np.nan
        else: # DOI is not np.nan
            doi = str(df["DOI"][ind]).strip()
        # print(doi)

        # get full_text_url, full_text_source
        if pmcid == pmcid: # pmcid is not np.nan
            full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
            full_text_source = "PMC"
        else: # pmcid is np.nan
            # PMC does not include this paper
            try:
                full_text_url = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["href"].strip()
                full_text_source = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["data-ga-action"].strip()
            except:
                full_text_url = np.nan
                full_text_source = np.nan
        # print(full_text_url)
        # print(full_text_source)
        
        # get pdf_url, pdf_source
        pdf_url = np.nan
        pdf_source = np.nan
                
        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["Title"][ind]).strip()],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_pubmed
# output_path = fpath.poten_litera_pubmed_processed
# # plib.clear_file(output_path)

# df = pd.read_csv(source_path, sep=',')
# print(df.shape)
# df = df[["DOI", "PMID", "PMCID", "Title"]]
# print(df.head(3))
# print(df.shape)

# print(df["DOI"].isnull().values.any())
# print(df["PMID"].isnull().values.any())
# print(df["PMCID"].isnull().values.any())
# print(df["Title"].isnull().values.any())
# # True, False, True, Flase
# # PMID, Title don't contain np.nan
# # DOI, PMCID contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_pubmed(source_path, output_path, start, end)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def preprocess_webofscience(source_path, output_path, start, end):
    print("Starting preprocessing search results from Web of Science...")
    
    df = pd.read_csv(source_path, sep=";")
    df = df[["DOI", "Pubmed Id", "Article Title"]]

    for ind in range(start, end):
        # sleep to avoid to be blocked
        time.sleep(random.randint(3, 5))
        # if(ind%50 == 0):
        #     time.sleep(random.randint(10,15)*10)
        
        # the columns Article Title don't contain np.nan
        # the columns DOI and PMID might contain np.nan
        # get pmid, doi
        if df["Pubmed Id"][ind] != df["Pubmed Id"][ind]: # Pubmed Id is np.nan
            if df["DOI"][ind] != df["DOI"][ind]: # DOI is np.nan
                doi = np.nan
                pmid = np.nan
            else: # DOI is not np.nan
                doi = str(df["DOI"][ind]).strip()
                pmid = plib.doi2pmid(doi)
        else: # Pubmed Id is not np.nan
            pmid = str(int(df["Pubmed Id"][ind])).strip()
            if df["DOI"][ind] != df["DOI"][ind]: # DOI is not np.nan
                doi = plib.pmid2doi(pmid)
            else: # DOI is not np.nan
                doi = str(df["DOI"][ind]).strip()
        
        # get pmcid, full_text_url, full_text_source
        if pmid != pmid: # pmid is np.nan
            pmcid = np.nan
            if doi != doi: # doi is np.nan
                full_text_url = np.nan
                full_text_source = np.nan
            else:
                full_text_url = "https://doi.org/" + str(doi).strip()
                full_text_source = "DOI"
        else: # pmid is not np.nan
            # request the webpage
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            # proxies = plib.get_proxies()
            soup = plib.request_webpage(url)
            # print(soup)

            # get pmcid
            try:
                pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
            except:
                pmcid = np.nan
            # print(pmcid)
            
            # get full_text_url, full_text_source
            if pmcid == pmcid:
                full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
                full_text_source = "PMC"
            else:
                try:
                    full_text_url = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["href"].strip()
                    full_text_source = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["data-ga-action"].strip()
                except:
                    full_text_url = np.nan
                    full_text_source = np.nan
        
        # get pdf_url, pdf_source
        pdf_url = np.nan
        pdf_source = np.nan

        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["Article Title"][ind]).strip()],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# # source_path = fpath.poten_litera_wos
# # output_path = fpath.poten_litera_wos_processed
# plib.clear_file(output_path)

# df = pd.read_csv(source_path, sep=';')
# df = df[["DOI", "Pubmed Id", "Article Title"]]
# print(df.head(3))
# print(df.shape)

# print(df["DOI"].isnull().values.any())
# print(df["Pubmed Id"].isnull().values.any())
# print(df["Article Title"].isnull().values.any())
# # True, True, False
# # Article Title don't contain np.nan
# # DOI, Pubmed Id contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code--------------------- 

# --------------------start of test code--------------------
# preprocess_webofscience(source_path, output_path, 0, 10)
# ---------------------end of test code--------------------- 

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=';')
# print(df.head(3))
# ---------------------end of test code---------------------  

In [ ]:
def preprocess_eupmc(source_path, output_path, start, end):
    print("Starting preprocessing search results from Europe PMC...")

    df = pd.read_csv(source_path, sep=",")
    df = df[["SOURCE", "DOI", "EXTERNAL_ID", "PMCID", "TITLE"]]

    for ind in range(start, end):
        # sleep to avoid to be blocked
        time.sleep(random.randint(1, 3))
        # if(ind%50 == 0):
        #     time.sleep(random.randint(10,15)*10)

        # get pmid, doi
        # SOURCE = {'PMC', 'MED', 'ETH', 'PPR'}
        if df["SOURCE"][ind] != "MED": # SOURCE is not "MED" 
            if df["DOI"][ind] != df["DOI"][ind]: # doi is np.nan
                doi = np.nan
                pmid = np.nan
            else:
                doi = str(df["DOI"][ind]).strip()
                pmid = plib.doi2pmid(doi)
        else: # SOURCE is "MED"
            # get doi, pmid
            if df["EXTERNAL_ID"][ind] != df["EXTERNAL_ID"][ind]: # EXTERNAL_ID is np.nan
                if df["DOI"][ind] != df["DOI"][ind](): # DOI is np.nan
                    doi = np.nan
                    pmid = np.nan
                else: # DOI is not np.nan
                    doi = str(df["DOI"][ind]).strip()
                    pmid = plib.doi2pmid(doi)
            else: # EXTERNAL_ID is not np.nan
                pmid = str(df["EXTERNAL_ID"][ind]).strip()
                if df["DOI"][ind] != df["DOI"][ind]: # DOI is np.nan
                    doi = plib.pmid2doi(pmid)
                else: # DOI is not np.nan
                    doi = str(df["DOI"][ind]).strip()
                
        # get pmcid, full_text_url, full_text_source
        if pmid != pmid: # pmid is np.nan
            pmcid = df["PMCID"][ind]
            if pmcid == pmcid: # pmcid is np.nan
                full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
                full_text_source = "PMC"
            elif doi == doi: # doi is not np.nan
                full_text_url = "https://doi.org/" + str(doi).strip()
                full_text_source = "DOI"
            else:
                full_text_url = np.nan
                full_text_source = np.nan
        else: # pmid is not np.nan
            # request the webpage
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            # proxies = plib.get_proxies()
            soup = plib.request_webpage(url)
            # print(soup)

            # get pmcid
            try:
                pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
            except:
                pmcid = np.nan
            # print(pmcid)
            
            # get full_text_url, full_text_source
            if pmcid == pmcid: # pmcid is not np.nan
                full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
                full_text_source = "PMC"
            else: # pmcid is not np.nan
                try:
                    full_text_url = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["href"].strip()
                    full_text_source = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["data-ga-action"].strip()
                except:
                    full_text_url = np.nan
                    full_text_source = np.nan
        
        # get pdf_url, pdf_source
        pdf_url = np.nan
        pdf_source = np.nan

        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["TITLE"][ind]).strip()],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_eupmc
# output_path = fpath.poten_litera_eupmc_processed
# # plib.clear_file(output_path)

# df = pd.read_csv(source_path, sep=',')
# df = df[["SOURCE", "DOI", "EXTERNAL_ID", "PMCID", "TITLE"]]
# print(df.head(3))
# print(df.shape)

# col_one_list = set(df['SOURCE'].tolist())
# print(col_one_list)
# # ['PMC', 'MED', 'ETH', 'PPR']

# print(df["SOURCE"].isnull().values.any())
# print(df["DOI"].isnull().values.any())
# print(df["EXTERNAL_ID"].isnull().values.any())
# print(df["PMCID"].isnull().values.any())
# print(df["TITLE"].isnull().values.any())
# # False, True, False, True, False
# # SOURCE, EXTERNAL_ID, Title don't contain np.nan
# # DOI, PMCID contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_eupmc(source_path, output_path, 0, 10)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def preprocess_google_shcolar_step1(source_path, output_path, start, end):
    print("Starting merging search results from Google Scholar...")

    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]

    for ind in range(start, end):
        # df["url_type"][ind]: {'[CITATION][C]', '[PDF][PDF]', '[BOOK][B]', nan, '[HTML][HTML]'}
        # we don't need citations and books, as they are not likely to include connecivity information
        if (df["url_type"][ind] == "[CITATION][C]") or (df["url_type"][ind] == "[BOOK][B]"):
            continue
        
        # if url or title doesn't exsit AND full_text_url doesn't exist
        if (df["url"][ind] != df["url"][ind]) or (df["title"][ind] != df["title"][ind]):
            continue
        
        # now every row has at least title and url, and the url_text = {"[PDF][PDF]", nan, "[HTML][HTML]"}
        if df["url_type"][ind] == "[PDF][PDF]":
            # full_text_type = {'[HTML]', nan, '[PDF]', 'UB'}
            if df["full_text_type"][ind] == "[HTML]":
                link = str(df["full_text_url"][ind]).strip()
                full_text_url, status_code  = plib.get_final_redirected_url(link)
                if full_text_url == full_text_url:
                    full_text_source = full_text_url.split("://")[1].split("/")[0]
                else:
                    full_text_source = np.nan
            else:
                full_text_url = np.nan
                full_text_source = np.nan
            # get pdf_url, pdf_source
            link = str(df["url"][ind]).strip()
            pdf_url, status_code  = plib.get_final_redirected_url(link)
            if pdf_url == pdf_url:
                pdf_source = pdf_url.split("://")[1].split("/")[0]
            else:
                pdf_source = np.nan
        else: # df["url_type"][ind] == nan or '[HTML][HTML]'
            link = str(df["url"][ind]).strip()
            full_text_url, status_code = plib.get_final_redirected_url(link)
            if full_text_url == full_text_url:
                full_text_source = full_text_url.split("://")[1].split("/")[0]
            else:
                full_text_source = np.nan
            # get pdf_url, pdf_source
            # full_text_type = {'[HTML]', nan, '[PDF]', 'UB'}
            if df["full_text_type"][ind] == "[PDF]":
                link = str(df["full_text_url"][ind]).strip()
                pdf_url, status_code  = plib.get_final_redirected_url(link)
                if pdf_url == pdf_url:
                    pdf_source = pdf_url.split("://")[1].split("/")[0]
                else:
                    pdf_source = np.nan
            else:
                pdf_url = np.nan
                pdf_source = np.nan
        
        columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "Title": [str(df["title"][ind]).strip()],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_gs
# output_path = fpath.poten_litera_gs_processed_step1
# # plib.clear_file(output_path)

# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]
# print(df.head(3))
# print(df.head)

# url_type = set(df['url_type'].tolist())
# print(url_type)
# # {'[CITATION][C]', '[PDF][PDF]', '[BOOK][B]', nan, '[HTML][HTML]'}
# full_text_type = set(df['full_text_type'].tolist())
# print(full_text_type)
# # {nan, 'UB', '[HTML]', '[PDF]'}
# full_text_source = set(df['full_text_source'].tolist())
# print(full_text_source)
# # {'ahajournals.org', 'lww.com', 'springer.com', 'academia.edu', 'plos.org', 'ieee.org', 'nature.com', 
# # 'mdpi.com', 'jpn.ca', 'uottawa.ca', nan, 'northwestern.edu', 'bmj.com', 'ekja.org', 'RWTH-Link', 'wiley.com', 
# # 'escholarship.org', 'nyu.edu', 'frontiersin.org', 'sciencedirect.com', 'eneuro.org', 'jneurosci.org', 
# # 'royalsocietypublishing.org', 'karger.com', 'harvard.edu', 'annualreviews.org', 'mcgill.ca', 
# # 'elifesciences.org', 'mirasmart.com', 'duke.edu', 'ucdavis.edu', 'physiology.org', 'cell.com', 
# # 'wustl.edu', 'epfl.ch', 'udc.es', 'psychiatryonline.org', 'jst.go.jp', 'core.ac.uk', 'rero.ch', 
# # 'zsp.com.pk', 'sagepub.com', 'europepmc.org', 'tandfonline.com', 'asahq.org', 'sonar.ch', 'koreamed.org', 
# # 'oup.com', 'science.org', 'scholarpedia.org', 'psu.edu', 'jordanbpeterson.com', 'pnas.org', 'uzh.ch', 'biorxiv.org', 
# # 'biomedcentral.com', 'umich.edu', 'ahuman.org', 'researchgate.net', 'ijpp.com', 'unav.edu', 'nih.gov', 'bu.edu'}
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # ["title", "url", "url_type", "full_text_url", "full_text_type", "full_text_source"]
# print(df["title"].isnull().any().any())
# print(df["url"].isnull().any().any())
# print(df["url_type"].isnull().any().any())
# print(df["full_text_url"].isnull().any().any())
# print(df["full_text_type"].isnull().any().any())
# print(df["full_text_source"].isnull().any().any())
# # True, True, True, True, True, True
# # title, url, url_type, full_text_url, full_text_type, full_text_source contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_google_shcolar_step1(source_path, output_path, 0, 1000)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def preprocess_google_shcolar_step2(source_path, output_path, start, end):
    print("Starting merging search results from Google Scholar...")

    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]

    for ind in range(start, end):
        # get doi from url
        if df["full_text_url"][ind] == df["full_text_url"][ind]: # there's a full_text_url
            url = str(df["full_text_url"][ind]).strip()
            source = str(df["full_text_source"][ind]).strip()
            info = plib.extract_info_from_webpage(url)
            doi = info["doi"]
            pmid = info["pmid"]
            pmcid = info["pmcid"]
        else:
            doi = np.nan
            pmid = np.nan
            pmcid = np.nan
        # # get pmid from DOI
        # if doi == doi: # there's doi
        #     pmid = plib.doi2pmid(doi)
        # else: # doi not found
        #     pmid = np.nan
        # # get pmcid, full_text_url, full_text_source
        # if pmid != pmid: # pmid is np.nan
        #     pmcid = np.nan
        #     if doi == doi: # doi is not np.nan
        #         full_text_url = "https://doi.org/" + str(doi).strip()
        #         full_text_source = "DOI"
        #     else:
        #         full_text_url = np.nan
        #         full_text_source = np.nan
        # else: # pmid is not np.nan
        #     # request the webpage
        #     url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
        #     # proxies = plib.get_proxies()
        #     soup = plib.request_webpage(url)
        #     # print(soup)

        #     # get pmcid
        #     try:
        #         pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
        #     except:
        #         pmcid = np.nan
        #     # print(pmcid)
            
        #     # get full_text_url, full_text_source
        #     if pmcid == pmcid: # pmcid is not np.nan
        #         full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
        #         full_text_source = "PMC"
        #     else: # pmcid is not np.nan
        #         try:
        #             full_text_url = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["href"].strip()
        #             full_text_source = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["data-ga-action"].strip()
        #         except:
        #             full_text_url = np.nan
        #             full_text_source = np.nan

        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [df["Title"][ind]],
            "full_text_url": [df["full_text_url"][ind]],
            "full_text_source": [df["full_text_source"][ind]],
            "pdf_url": [df["pdf_url"][ind]],
            "pdf_source": [df["pdf_source"][ind]]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(doi)
        if doi != doi:
            print([df["full_text_url"][ind]])
        print(ind)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_gs_processed_step1
# output_path = fpath.poten_litera_gs_processed_step2
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
# print(df.head(3))
# print(df.shape)
# # # (905, 5)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# full_text_source = set(df['full_text_source'].tolist())
# print(full_text_source)
# # {'www.frontiersin.org', 'www.elibrary.ru', 'orca.cardiff.ac.uk', 'www.jneurosci.org', 
# # 'europepmc.org', 'www.theses.fr', 'www.biorxiv.org', 'submissions.mirasmart.com', 
# # 'royalsocietypublishing.org', 'www.science.org', 'thejns.org', 
# # 'escholarship.mcgill.ca', 'www.cambridge.org', 'movementdisorders.onlinelibrary.wiley.com', 
# # 'www.ahajournals.org', 'books.google.de', 'www.mdpi.com', 'www.sciencedirect.com', 
# # 'ieeexplore.ieee.org', 'academic.oup.com', 'www.pnas.org', 'physoc.onlinelibrary.wiley.com', 
# # 'www.jstage.jst.go.jp', 'wakespace.lib.wfu.edu', 'elibrary.ru', 'www.cabdirect.org', 
# # 'www.tandfonline.com', 'www.jpn.ca', 'jpet.aspetjournals.org', 'onlinelibrary.wiley.com', 
# # 'open.bu.edu', 'tbiomed.biomedcentral.com', 'www.liebertpub.com', 'journals.lww.com', 
# # 'agro.icm.edu.pl', 'ekja.org', 'analyticalsciencejournals.onlinelibrary.wiley.com', 
# # 'n.neurology.org', 'pubs.asahq.org', 'journals.sagepub.com', 'neuro.psychiatryonline.org', 
# # 'karger.com', 'nyaspubs.onlinelibrary.wiley.com', 'pure.mpg.de', 'elifesciences.org', 
# # 'link.springer.com', 'psycnet.apa.org', 'jnnp.bmj.com', 'www.degruyter.com', 'ajp.psychiatryonline.org', 
# # 'journals.physiology.org', 'www.nature.com', 'www.jstor.org', 'var.scholarpedia.org', 'www.eneuro.org', 
# # 'journals.plos.org', 'www.cell.com', 'www.ncbi.nlm.nih.gov', 'www.taylorfrancis.com', 
# # 'bmcneurosci.biomedcentral.com', nan, 'jamanetwork.com'}
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
# print(df["Title"].isnull().any().any())
# print(df["full_text_url"].isnull().any().any())
# print(df["full_text_source"].isnull().any().any())
# print(df["pdf_url"].isnull().any().any())
# print(df["pdf_source"].isnull().any().any())
# # False, True, True, True, True
# # full_text_url, full_text_source, pdf_url, pdf_source contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_google_shcolar_step2(source_path, output_path, 0, 905)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def preprocess_seed_paper_spanning(source_path, output_path):
    print("Starting preprocessing search results from spanning citations of seed paper...")
    return True
# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [ ]:
def preprocess_cocomac_paper(source_path, output_path):
    print("Starting preprocessing search results from CoCoMac papers...")
    return True
# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [ ]:
def combine(input, output_path):
    # combine all results
    df = pd.DataFrame()
    for search_result in input:
        df_single = pd.read_csv(search_result, header=None, sep = ",")
        # df = df.append(df_single, ignore_index=True, sort=False)
        df = pd.concat([df, df_single], ignore_index=True, sort=False)
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
    df.to_csv(output_path, header=False, index=False)
# --------------------start of test code--------------------
# gos = fpath.poten_litera_gs_processed_step2
# wos = fpath.poten_litera_wos_processed
# pubmed = fpath.poten_litera_pubmed_processed
# eupmc = fpath.poten_litera_eupmc_processed
# input = [gos, wos, pubmed, eupmc]
# output_path = fpath.poten_litera_combined
# # plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# combine(input, output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# print(df.shape)
# # (14627, 8)
# ---------------------end of test code---------------------

In [ ]:
def fill_in_missing(input_path, output_path, start, end):
    df = pd.read_csv(input_path, header=None, sep = ",")
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
    
    # fill in elements that are missing
    for ind in range(start, end):
        # initialzie
        doi = np.nan
        pmid = np.nan
        pmcid = np.nan
        title = str(df["Title"][ind]).strip()
        full_text_link = np.nan
        pdf_link = np.nan

        # doi, pmid
        if df["DOI"][ind] == df["DOI"][ind]: # DOI -> PMID
            doi = str(df["DOI"][ind]).strip().lower()
            # print(doi)
            if df["PMID"][ind] == df["PMID"][ind]:
                pmid = str(df["PMID"][ind]).strip()
                # print(pmid)
            else:
                pmid = plib.doi2pmid(doi)
                # print(pmid)
                if pmid != pmid:
                    pmid_cadidate = plib.title2pmid(title)
                    # print(pmid_cadidate)
                    if pmid_cadidate == pmid_cadidate:   
                        doi_validate, a = plib.pmid2doi_pmcid(pmid_cadidate)
                        if doi_validate == doi_validate:
                            doi_validate = doi_validate.lower()
                            if doi_validate == doi:
                                pmid = pmid_cadidate
                                # print(pmid)
        elif df["PMID"][ind] == df["PMID"][ind]: # PMID -> DOI
            pmid = str(df["PMID"][ind]).strip()
            # print(pmid)
            doi, pmcid = plib.pmid2doi_pmcid(str(pmid).strip())
            # print(doi)
        elif df["PMCID"][ind] == df["PMCID"][ind]: # PMCID -> DOI, PMID
            pmcid = str(df["PMCID"][ind]).strip()
            doi, pmid = plib.pmcid2doi_pmid(pmcid)
            # print(doi)
            # print(pmid)
        else:
            doi = np.nan
            pmid = np.nan
        # print(doi)
        # print(pmid)
        
        # pmcid
        if df["PMCID"][ind] == df["PMCID"][ind]:
            pmcid = str(df["PMCID"][ind]).strip()
        elif pmid == pmid:
            doi_1, pmcid = plib.pmid2doi_pmcid(str(pmid).strip())
        else:
            pmcid = np.nan
        # print(pmcid)

        # full_text_link
        if pmcid == pmcid:
            full_text_link = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
        elif doi == doi:
            full_text_link = plib.get_final_redirected_url(str("https://doi.org/" + doi).strip())
        elif df["full_text_url"][ind] == df["full_text_url"][ind]:
            full_text_link = plib.get_final_redirected_url(df["full_text_url"][ind])
        else:
            full_text_link = np.nan
        # print(full_text_link)

        # pdf_link
        if df["pdf_url"][ind] == df["pdf_url"][ind]:
            pdf_link = plib.get_final_redirected_url(str(df["pdf_url"][ind]).strip())
        else:
            pdf_link = np.nan
        # print(pdf_link)
    
        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
        row = {
            "DOI": [doi.lower()],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [title],
            "full_text_link": [full_text_link],
            "pdf_link": [pdf_link],
        }

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")

        print(ind)
# --------------------start of test code--------------------
# input_path = fpath.poten_litera_combined
# output_path = fpath.poten_litera_filled
# # plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# fill_in_missing(input_path, output_path, 0, 14627)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def merge_remove_dupli(input_path, output_path, identifiers): 
    df = pd.read_csv(input_path, header=None, sep = ",")
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]

    # remove all duplicates
    for identifier in identifiers:
        remove_dup_by = identifier
        df = df[df[remove_dup_by].isnull() | ~df[df[remove_dup_by].notnull()].duplicated(subset=remove_dup_by, keep='first')]
        # df = df.drop_duplicates(subset=['DOI'])
        # df = df.drop_duplicates(subset=['PMID'])
        # df = df.drop_duplicates(subset=['PMCID'])

    # reset index
    df.reset_index(drop=True, inplace=True)
    
    df.to_csv(output_path, header=False, index=False)
    print("Duplication in the potential related literature removed.")
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_ids_filled
# output_path = fpath.poten_litra_filtered
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# merge all search results
# identifiers = ["DOI", "PMID", "PMCID"]
# merge_remove_dupli(source_path, output_path, identifiers)
# ---------------------end of test code---------------------

In [ ]:
def full_text_link_filling(input_path, output_path, start, end):
    df = pd.read_csv(input_path, header=None, sep = ",")
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
    
    for ind in range(start, end):
        pmcid = df.at[ind, "PMCID"]
        doi = df.at[ind, "DOI"]
        full_text_link = np.nan

        # get full text link
        if pmcid == pmcid:
            url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + str(pmcid).strip() + "/"
            try:
                full_text_link, status_code = plib.get_final_redirected_url(url)
                if status_code == 403:
                    link = "https://pubmed.ncbi.nlm.nih.gov/" + str(int(df.at[ind, "PMID"])).strip() + "/"
                    soup = plib.request_webpage(link)
                    link = soup.find("div", {"class": "full-text-links-list"}).find("a", {"class": "link-item dialog-focus"})["href"]
                    full_text_link, status_code = plib.get_final_redirected_url(link)
            except:
                raise Exception("Error when trying to get final redirected url from", url)
        
        if full_text_link != full_text_link and df.at[ind, "full_text_link"] == df.at[ind, "full_text_link"]:
            try:
                full_text_link, status_code = plib.get_final_redirected_url(df.at[ind, "full_text_link"])
            except:
                raise Exception("Error when trying to get final redirected url from", df.at[ind, "full_text_link"])
        
        if full_text_link != full_text_link and doi == doi:
            url = "https://doi.org/" + str(doi).strip().lower()
            try:
                full_text_link, status_code = plib.get_final_redirected_url(url)
            except:
                raise Exception("Error when trying to get final redirected url from", url)
        
        if  full_text_link != full_text_link and df["pdf_link"][ind] == df["pdf_link"][ind]:
            full_text_link = np.nan
        
        if full_text_link != full_text_link:
            continue
        # print(full_text_link)

        # get full text source
        if full_text_link == full_text_link:
            full_text_source = full_text_link.split("://")[1].split("/")[0]
        else:
            continue
        
        if doi == doi:
            doi = doi.lower()
        
        if df.at[ind, "PMID"] == df.at[ind, "PMID"]:
            pmid = str(int(df.at[ind, "PMID"])).strip()
        else:
            pmid = np.nan
        
        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "full_text_source", "pdf_link"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [df.at[ind, "Title"]],
            "full_text_link": [full_text_link],
            "full_text_source": [full_text_source],
            "pdf_link": [df.at[ind, "pdf_link"]]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)

<h3> Main program: </h3> 

In [ ]:
# # preprocess search results from PubMed

# source_path = fpath.poten_litera_pubmed
# output_path = fpath.poten_litera_pubmed_processed

# # clear the file
# plib.clear_file(output_path)

# # preprocess search results from PubMed
# # 2606 results
# preprocess_pubmed(source_path, output_path, columns, 2565, 2606)
# print("preprocessing results from PubMed succeeded!")
# # print("Attention! Something went wrong when preprocessing results from PubMed!")

In [ ]:
# # clear the file
# plib.clear_file(fpath.poten_litera_wos)

# # combine the 2 files of search results from web of science
# source_path_1 = fpath.poten_litera_wos_1
# source_path_2 = fpath.poten_litera_wos_2
# df_1 = pd.read_csv(source_path_1, sep=';')
# df_2 = pd.read_csv(source_path_2, sep=';')
# df_1.to_csv(fpath.poten_litera_wos, header=True, index=False, sep=";")
# df_2.to_csv(fpath.poten_litera_wos, mode="a", header=False, index=False, sep=";")
# --------------------start of test code--------------------
# df = pd.read_csv(fpath.poten_litera_wos, sep=';')
# print(df.head(3))
# print(df.shape)
# (1976, 72)
# ---------------------end of test code---------------------

In [ ]:
# # preprocess search results from Web of Science

# source_path = fpath.poten_litera_wos
# output_path = fpath.poten_litera_wos_processed

# # clear the file
# # plib.clear_file(output_path)

# # preprocess search results from Web of Science
# # 1976 results
# preprocess_webofscience(source_path, output_path, columns, 0, 1976)
# print("preprocessing results from Web of Science succeeded!")
# # print("Attention! Something went wrong when preprocessing results from Web of Science!")

In [ ]:
# # preprocess search results from Europe PMC

# source_path = fpath.poten_litera_eupmc
# output_path = fpath.poten_litera_eupmc_processed

# # clear the file
# # plib.clear_file(output_path)

# # preprocess search results from Europe PMC
# preprocess_eupmc(source_path, output_path, columns, 0, 9140)
# # 9140 results
# print("preprocessing results from Europe PMC succeeded!")
# # print("Attention! Something went wrong when preprocessing results from Europe PMC!")

In [ ]:
# # preprocess search results from Google Scholar step 1

# source_path = fpath.poten_litera_gs
# output_path = fpath.poten_litera_gs_processed_step1

# # clear the file
# # plib.clear_file(output_path)

# # preprocess search results from Google Scholar
# preprocess_google_shcolar_step1(source_path, output_path, 0, 1000)
# # 905 results
# print("step 1 of preprocessing results from Google Scholar succeeded!")
# # print("Attention! Something went wrong when preprocessing results from Google Scholar step 1!")

In [ ]:
# # preprocess search results from Google Scholar step 2

# source_path = fpath.poten_litera_gs_processed_step1
# output_path = fpath.poten_litera_gs_processed_step2

# # clear the file
# # plib.clear_file(output_path)

# # preprocess search results from Google Scholar
# preprocess_google_shcolar_step2(source_path, output_path, 0, 905)
# # 905 results
# print("step 2 of preprocessing results from Google Scholar succeeded!")
# # print("Attention! Something went wrong when preprocessing results from Google Scholar step 2!")

In [ ]:
# # preprocess search results from spanning citations of seed paper

# preprocess_seed_paper_spanning(source_path, output_path, columns):
# print("preprocessing results from spanning citations of seed papers succeeded!")
# # print("Attention! Something went wrong when preprocessing results from spanning citations of seed papers!")

In [ ]:
# # preprocess search results from CoCoMac papers

# preprocess_cocomac_paper(source_path, output_path, columns)
# print("preprocessing results from CoCoMac papers succeeded!")
# # print("Attention! Something went wrong when preprocessing results from CoCoMac papers!")

In [ ]:
# # combine all search results

# gos = fpath.poten_litera_gs_processed_step2
# wos = fpath.poten_litera_wos_processed
# pubmed = fpath.poten_litera_pubmed_processed
# eupmc = fpath.poten_litera_eupmc_processed
# input = [gos, wos, pubmed, eupmc]
# output_path = fpath.poten_litera_combined

# # clear the file
# # plib.clear_file(output_path)

# combine(input, output_path)
# # (14627, 8)
# print("Combining all search results succeeded!")
# # # print("Attention! Something went wrong when combining all search results!")

In [ ]:
# # fill in elements missed, remove duplciations based on identifiers in the potential related literature

# input_path = fpath.poten_litera_combined
# output_path = fpath.poten_litera_ids_filled


# # plib.clear_file(output_path)

# fill_in_missing(input_path, output_path, 0, 14627)
# print("Filling in missing elements succeeded!")
# # print("Attention! Something went wrong when filling in missing elements!")

In [ ]:
# # merge all search results and remove duplication by identifiers

# source_path = fpath.poten_litera_ids_filled
# output_path = fpath.poten_litra_filtered

# # clear the file
# plib.clear_file(output_path)

# # merge all search results
# identifiers = ["DOI", "PMID", "PMCID"]
# merge_remove_dupli(source_path, output_path, identifiers)

In [ ]:
# source_path = fpath.poten_litra_filtered
# df = pd.read_csv(source_path, header=None, sep=',')
# print(df.shape)
# # (11021, 6)

In [ ]:
# fill in full_text_link

source_path = fpath.poten_litra_filtered
output_path = fpath.poten_litera_ids_ftl_filled

# clear the file
plib.clear_file(output_path)

# merge all search results
full_text_link_filling(source_path, output_path, 0, 11021)

In [ ]:
# # reset the index of the dataframe and save it to a csv file
# source_path = fpath.poten_litera_ids_ftl_filled
# output_path = fpath.poten_litera_ids_ftl_filled
# df = pd.read_csv(source_path, header=None, sep=',')
# df.reset_index(drop=True, inplace=True)
# df.to_csv(output_path, header=False, index=False)

In [ ]:
# source_path = fpath.poten_litera_ids_ftl_filled
# df = pd.read_csv(source_path, header=None, sep=',')
# # print(df.head(5))
# print(df.shape)
# # (10769, 7)

In [ ]:
# check all possible full_text_source
input_path = fpath.poten_litera_ids_ftl_filled
df = pd.read_csv(input_path, header=None, sep=",")
columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "full_text_source", "pdf_link"]
df.columns = columns

print(df.head(3))
print(df.shape)
# (10769, 7)

full_text_source = set(df['full_text_source'].tolist())
print(full_text_source)
# {'pure.mpg.de', 'direct.mit.edu', 'pubs.acs.org', 'thejns.org', 'www.microbiologyresearch.org', 'www.ncbi.nlm.nih.gov', 
#  'journals.sagepub.com', 'journals.physiology.org', 'www.thieme-connect.de', 'www.jstage.jst.go.jp', 'www.rbojournal.org', 
#  'www.annualreviews.org', 'var.scholarpedia.org', 'ujms.net', 'papers.ssrn.com', 'www.degruyter.com', 'jamanetwork.com', 
#  'escholarship.mcgill.ca', 'www.tandfonline.com', 'wakespace.lib.wfu.edu', 'www.taylorfrancis.com', 'content.iospress.com:443', 
#  'www.cambridge.org', 'n.neurology.org', 'journals.biologists.com', 'www.nature.com', 'pubs.aip.org', 'books.google.de', 
#  'linkinghub.elsevier.com', 'academic.oup.com', 'link.springer.com', 'karger.com', 'neurologia.com', 'onlinelibrary.wiley.com', 
#  'www.ajtmh.org', 'iovs.arvojournals.org', 'elibrary.ru', 'psycnet.apa.org:443', 'journals.aps.org', 'royalsocietypublishing.org', 
#  'jpet.aspetjournals.org', 'www.biorxiv.org', 'ieeexplore.ieee.org', 'journals.lww.com', 'ekja.org', 'open.bu.edu', 
#  'www.cabdirect.org', 'www.elibrary.ru', 'jnm.snmjournals.org', 'www.architalbiol.org', 'www.imrpress.com', 
#  'neuro.psychiatryonline.org', 'submissions.mirasmart.com', 'pubs.asahq.org', 'europepmc.org', 'www.ahajournals.org', 
#  'www.science.org', 'nrc-prod.literatumonline.com', 'pharmrev.aspetjournals.org', 'www.liebertpub.com', 'opg.optica.org', 
#  'www.ingentaconnect.com', 'symposium.cshlp.org', 'ajp.psychiatryonline.org', 'webview.isho.jp', 'www.theses.fr', 
#  'www.worldscientific.com'}

# ["DOI", "PMID", "PMCID", "Title", "full_text_link", "full_text_source", "pdf_link"]
print(df["DOI"].isnull().any().any()) # True
print(df["PMID"].isnull().any().any()) # True
print(df["PMCID"].isnull().any().any()) # True
print(df["Title"].isnull().any().any()) # False
print(df["full_text_link"].isnull().any().any()) # False
print(df["full_text_source"].isnull().any().any()) # False
print(df["pdf_link"].isnull().any().any()) # True

print(df["DOI"].dtypes) # object
print(df["PMID"].dtypes) # float64
print(df["PMCID"].dtypes) # object
print(df["Title"].dtypes) # object
print(df["full_text_link"].dtypes) # object
print(df["full_text_source"].dtypes) # object
print(df["pdf_link"].dtypes) # object

In [ ]:
# websites_hosts
# {'pure.mpg.de', 'direct.mit.edu', 'pubs.acs.org', 'thejns.org', 'www.microbiologyresearch.org', 'www.ncbi.nlm.nih.gov', 
#  'journals.sagepub.com', 'journals.physiology.org', 'www.thieme-connect.de', 'www.jstage.jst.go.jp', 'www.rbojournal.org', 
#  'www.annualreviews.org', 'var.scholarpedia.org', 'ujms.net', 'papers.ssrn.com', 'www.degruyter.com', 'jamanetwork.com', 
#  'escholarship.mcgill.ca', 'www.tandfonline.com', 'wakespace.lib.wfu.edu', 'www.taylorfrancis.com', 'content.iospress.com:443', 
#  'www.cambridge.org', 'n.neurology.org', 'journals.biologists.com', 'www.nature.com', 'pubs.aip.org', 'books.google.de', 
#  'linkinghub.elsevier.com', 'academic.oup.com', 'link.springer.com', 'karger.com', 'neurologia.com', 'onlinelibrary.wiley.com', 
#  'www.ajtmh.org', 'iovs.arvojournals.org', 'elibrary.ru', 'psycnet.apa.org:443', 'journals.aps.org', 'royalsocietypublishing.org', 
#  'jpet.aspetjournals.org', 'www.biorxiv.org', 'ieeexplore.ieee.org', 'journals.lww.com', 'ekja.org', 'open.bu.edu', 
#  'www.cabdirect.org', 'www.elibrary.ru', 'jnm.snmjournals.org', 'www.architalbiol.org', 'www.imrpress.com', 
#  'neuro.psychiatryonline.org', 'submissions.mirasmart.com', 'pubs.asahq.org', 'europepmc.org', 'www.ahajournals.org', 
#  'www.science.org', 'nrc-prod.literatumonline.com', 'pharmrev.aspetjournals.org', 'www.liebertpub.com', 'opg.optica.org', 
#  'www.ingentaconnect.com', 'symposium.cshlp.org', 'ajp.psychiatryonline.org', 'webview.isho.jp', 'www.theses.fr', 
#  'www.worldscientific.com'}
websites_hosts = [
    'karger.com', 'rbojournal.org', 'sagepub.com', 'neurology.org', 'asahq.org', 'aspetjournals.org', 'thieme-connect.de', 
    'taylorfrancis.com', 'lww.com', 'neurologia.com', 'ekja.org', 'www.imrpress.com', 'europepmc.org', 'springer.com', 
    'theses.fr', 'ieee.org', 'ssrn.com', 'nature.com', 'liebertpub.com', 'oup.com', 'open.bu.edu', 'journals.biologists.com', 
    'aip.org', 'mpg.de', 'lib.wfu.edu', 'cambridge.org', 'literatumonline.com', 'acs.org', 'scholarpedia.org', 'isho.jp', 
    'mirasmart.com', 'jstage.jst.go.jp', 'psychiatryonline.org', 'psycnet.apa.org', 'thejns.org', 'microbiologyresearch.org', 
    'wiley.com', 'snmjournals.org', 'degruyter.com', 'worldscientific.com', 'opg.optica.org', 'science.org', 'aps.org', 
    'ujms.net', 'mit.edu', 'biorxiv.org','annualreviews.org', 'elibrary.ru', 'www.ingentaconnect.com', 'mcgill.ca', 
    'symposium.cshlp.org', 'architalbiol.org', 'arvojournals.org', 'jamanetwork.com', 'elsevier.com', 'ncbi.nlm.nih.gov', 
    'cabdirect.org', 'books.google.de', 'iospress.com', 'tandfonline.com', 'ajtmh.org', 'royalsocietypublishing.org', 
    'ahajournals.org', 'physiology.org']
# --------------------start of test code--------------------
if len(websites_hosts) == len(set(websites_hosts)):
    print("There are no duplicates in the list.")
else:
    print("There are duplicates in the list.")
# ---------------------end of test code---------------------

In [ ]:
# sort the websites by the number of articles they have
input_path = fpath.poten_litera_ids_ftl_filled
df = pd.read_csv(input_path, header=None, sep=",")
columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "full_text_source", "pdf_link"]
df.columns = columns
func_dict = {website: 0 for website in websites_hosts}
# print(func_dict)

for ind in df.index:
    for website in websites_hosts:
        if website in df.loc[ind, "full_text_source"]:
            func_dict[website] += 1
            break

# Sort dictionary by values
sorted_dict = dict(sorted(func_dict.items(), key=lambda item: item[1], reverse=True))
print(sorted_dict)
# {'ncbi.nlm.nih.gov': 7886, 'elsevier.com': 1019, 'wiley.com': 696, 'springer.com': 285, 'physiology.org': 205, 
#  'oup.com': 152, 'cambridge.org': 74, 'karger.com': 53, 'lww.com': 49, 'nature.com': 44, 'science.org': 30, 
#  'tandfonline.com': 29, 'sagepub.com': 21, 'jamanetwork.com': 20, 'neurology.org': 16, 'biorxiv.org': 15, 
#  'royalsocietypublishing.org': 13, 'psycnet.apa.org': 12, 'arvojournals.org': 12, 'jstage.jst.go.jp': 11, 
#  'psychiatryonline.org': 11, 'europepmc.org': 10, 'mit.edu': 10, 'thejns.org': 8, 'annualreviews.org': 8, 
#  'snmjournals.org': 7, 'aspetjournals.org': 6, 'elibrary.ru': 5, 'books.google.de': 5, 'architalbiol.org': 4, 
#  'ahajournals.org': 4, 'liebertpub.com': 3, 'acs.org': 3, 'degruyter.com': 3, 'worldscientific.com': 3, 
#  'iospress.com': 3, 'asahq.org': 2, 'thieme-connect.de': 2, 'neurologia.com': 2, 'mpg.de': 2, 'opg.optica.org': 2, 
#  'mcgill.ca': 2, 'rbojournal.org': 1, 'taylorfrancis.com': 1, 'ekja.org': 1, 'www.imrpress.com': 1, 'theses.fr': 1, 
#  'ieee.org': 1, 'ssrn.com': 1, 'open.bu.edu': 1, 'journals.biologists.com': 1, 'aip.org': 1, 'lib.wfu.edu': 1, 
#  'literatumonline.com': 1, 'scholarpedia.org': 1, 'isho.jp': 1, 'mirasmart.com': 1, 'microbiologyresearch.org': 1, 
#  'aps.org': 1, 'ujms.net': 1, 'www.ingentaconnect.com': 1, 'symposium.cshlp.org': 1, 'cabdirect.org': 1, 'ajtmh.org': 1}

non_zero_keys = [key for key, value in sorted_dict.items() if value != 0]
print(non_zero_keys)
# ['ncbi.nlm.nih.gov', 'elsevier.com', 'wiley.com', 'springer.com', 'physiology.org', 'oup.com', 'cambridge.org', 
#  'karger.com', 'lww.com', 'nature.com', 'science.org', 'tandfonline.com', 'sagepub.com', 'jamanetwork.com', 
#  'neurology.org', 'biorxiv.org', 'royalsocietypublishing.org', 'psycnet.apa.org', 'arvojournals.org', 'jstage.jst.go.jp', 
#  'psychiatryonline.org', 'europepmc.org', 'mit.edu', 'thejns.org', 'annualreviews.org', 'snmjournals.org', 
#  'aspetjournals.org', 'elibrary.ru', 'books.google.de', 'architalbiol.org', 'ahajournals.org', 'liebertpub.com', 
#  'acs.org', 'degruyter.com', 'worldscientific.com', 'iospress.com', 'asahq.org', 'thieme-connect.de', 'neurologia.com', 
#  'mpg.de', 'opg.optica.org', 'mcgill.ca', 'rbojournal.org', 'taylorfrancis.com', 'ekja.org', 'www.imrpress.com', 
#  'theses.fr', 'ieee.org', 'ssrn.com', 'open.bu.edu', 'journals.biologists.com', 'aip.org', 'lib.wfu.edu', 
#  'literatumonline.com', 'scholarpedia.org', 'isho.jp', 'mirasmart.com', 'microbiologyresearch.org', 'aps.org', 
#  'ujms.net', 'www.ingentaconnect.com', 'symposium.cshlp.org', 'cabdirect.org', 'ajtmh.org']

In [ ]:
# websites
websites = [
    'ncbi.nlm.nih.gov', 'elsevier.com', 'wiley.com', 'springer.com', 'physiology.org', 'oup.com', 
    'cambridge.org', 'karger.com', 'lww.com', 'nature.com', 'science.org', 'tandfonline.com', 
    'sagepub.com', 'jamanetwork.com', 'neurology.org', 'biorxiv.org', 'royalsocietypublishing.org', 
    'psycnet.apa.org', 'arvojournals.org', 'jstage.jst.go.jp', 'psychiatryonline.org', 'europepmc.org', 
    'mit.edu', 'thejns.org', 'annualreviews.org', 'snmjournals.org', 'aspetjournals.org', 'elibrary.ru', 
    'books.google.de', 'architalbiol.org', 'ahajournals.org', 'liebertpub.com', 'acs.org', 'degruyter.com', 
    'worldscientific.com', 'iospress.com', 'asahq.org', 'thieme-connect.de', 'neurologia.com', 'mpg.de', 
    'opg.optica.org', 'mcgill.ca', 'rbojournal.org', 'taylorfrancis.com', 'ekja.org', 'www.imrpress.com', 
    'theses.fr', 'ieee.org', 'ssrn.com', 'open.bu.edu', 'journals.biologists.com', 'aip.org', 'lib.wfu.edu', 
    'literatumonline.com', 'scholarpedia.org', 'isho.jp', 'mirasmart.com', 'microbiologyresearch.org', 
    'aps.org', 'ujms.net', 'www.ingentaconnect.com', 'symposium.cshlp.org', 'cabdirect.org', 'ajtmh.org'
]
# --------------------start of test code--------------------
if len(websites) == len(websites_hosts):
    print('The number of websites is correct')
# ---------------------end of test code---------------------

<h3> Next step: automatic filtering </h3>